In [36]:
#!pip install chromadb sentence-transformers openai pandas numpy matplotlib pillow scikit-learn requests


In [37]:
# from google.colab import files

# uploaded = files.upload()  # Upload CSV files manually if running in Google Colab

import pandas as pd

# Load CSV datasets
email_details = pd.read_csv("./CSV/email_thread_details.csv")
email_summaries = pd.read_csv("./CSV/email_thread_summaries.csv")
fashion_data = pd.read_csv("./CSV/Fashion Dataset v2.csv")


In [38]:
#pip install chromadb

In [39]:
pip install --upgrade openai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-1.64.0-py3-none-any.whl (472 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [40]:
from openai import OpenAI

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from PIL import Image
from io import BytesIO
import openai

try:
    import chromadb
    from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
except ImportError:
    raise ImportError('ChromaDB package not found. Install using: pip install chromadb')

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")

# Define embedding function
embedding_function = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

# Create collections
email_details_collection = chroma_client.get_or_create_collection(name="email_details", embedding_function=embedding_function)
email_summaries_collection = chroma_client.get_or_create_collection(name="email_summaries", embedding_function=embedding_function)
fashion_collection = chroma_client.get_or_create_collection(name="fashion_data", embedding_function=embedding_function)

# Function to insert data into ChromaDB
def insert_into_chromadb(collection, df, id_column, text_column):
    for index, row in df.iterrows():
        collection.add(
            ids=[str(row[id_column])],
            documents=[str(row[text_column])],
            metadatas=[row.to_dict()]
        )

# Insert email details into ChromaDB
insert_into_chromadb(email_details_collection, email_details, "thread_id", "body")

# Insert email summaries into ChromaDB
insert_into_chromadb(email_summaries_collection, email_summaries, "thread_id", "summary")

# Insert fashion data into ChromaDB
insert_into_chromadb(fashion_collection, fashion_data, "p_id", "name")

# Function to perform search in ChromaDB
def search_in_chromadb(collection, query, top_k=5):
    results = collection.query(
        query_texts=[query],
        n_results=top_k
    )
    return results["metadatas"][0]



Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 1
Insert of existing embedding ID: 1
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 2
Insert of existing embedding ID: 2
Add of existing embedding ID: 3
Insert of existing embedding ID: 3
Add of existing embedding ID: 3
Insert of existing embedding ID: 3
Add of existing embedding ID: 3
Insert of existing embedding I

In [42]:
pip install openai==0.28

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.64.0
    Uninstalling openai-1.64.0:
      Successfully uninstalled openai-1.64.0
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from openai import OpenAI
import os

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = ""

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # Ensure API key is set


In [ ]:





# Function to generate response using OpenAI API
def generate_response(context, query):
    role = "You are an AI assistant handling diverse information retrieval tasks."
    task = "Answer the query accurately using the provided context."

    
    response = client.chat.completions.create(
        model="gpt-4",
        messages = [
        {"role": "system", "content": f"{role} {task}"},
        {"role": "user", "content": f"Query: {query}\nContext: {context}"}
    ]
    )
    print(response.choices[0].message.content)  # Correct way to access content


# Function to display images from URLs
def display_images(results):
    fig, axes = plt.subplots(1, min(5, len(results)), figsize=(15, 5))
    if len(results) == 1:
        axes = [axes]

    for ax, row in zip(axes, results):
        try:
            response = requests.get(row["img"], timeout=5)
            img = Image.open(BytesIO(response.content))
            ax.imshow(img)
            ax.set_title(row["name"], fontsize=10)
            ax.axis("off")
        except Exception:
            ax.set_title("Image Not Available", fontsize=10)
            ax.axis("off")

    plt.show()

# Sample Queries

queries = [
    ("Find emails discussing budget approvals.", email_details_collection),
    ("Locate email summaries about product launches.", email_summaries_collection),
    ("Are there any Natural gas deals.", email_summaries_collection),
    ("Find stylish summer dresses for women.", fashion_collection),
    ("Recommend formal shoes for office wear.", fashion_collection),
    ("Suggest trendy winter jackets.", fashion_collection),
    ("Suggest Embroidery thread work Kurta.", fashion_collection),
]

# Retrieve results and generate responses with images (for fashion queries)
for query, collection in queries:
    results = search_in_chromadb(collection, query)
    response = generate_response(str(results), query)
    
    print(f"\nQuery: {query}\nResponse: {response}\n")
    
    if "imageUrl" in results[0]:
        display_images(results)

Based on the provided email context, the emails which discuss budget approvals include:

1. Email from Dawn Derr with the subject "Operating & Strategic Plan 2001 - 2003". It discusses the process for submitting budgets for specific Cost Centers.

2. Email from Derrick Jr., James JDERRIC, with the subject "FW: <<Concur Expense Document>> - various", discusses the approval status of an expense report.

3. Email from Derrick Jr., James JDERRIC, with the subject "FW: <<Concur Expense Document>> - Lunch" where the approval status of a lunch expense report is discussed.

4. Email from Derrick Jr., James JDERRIC, with the subject "FW: <<Concur Expense Document>> - Summer Clerk Lunch", details about the approval of a lunch expense report for a summer clerk is discussed.

Query: Find emails discussing budget approvals.
Response: None

The thread that talks about product launches is: {'summary': "The email thread discusses the finalization of product descriptions for new hub products. Dale sugg